In [33]:
using LinearAlgebra, Plots, Random, Distributions, Optim

First, let us create a NamedTuple with our parameters.

In [32]:
params = (; β = 0.72, # bargaining_power
            r = 0.012, # discount rate
            λ = 4, # arrival rate
            z = 0.4, # value of leisure
            p_star = 1., # productivity
            s_star = 0.1, # separation rate
            c = 0.213, # cost of vacancy
            σ = 0.0165, # standard deviation 
            γ = 0.004, # autoregressive coefficient
            n = 2001, # gridpoints
            lb = -1., # bounds for grid
            ub = 1., # bounds for grid
            mc = 1.355, # constant term in matching function
            me = -0.72, # exponent in matching function
            max_iter  = 1000 )

(β = 0.72, r = 0.012, λ = 4, z = 0.4, p_star = 1.0, s_star = 0.1, c = 0.213, σ = 0.0165, γ = 0.004, n = 2001, lb = -1.0, ub = 1.0, mc = 1.355, me = -0.72, max_iter = 1000)

In [25]:
function matching_function(grid, params)
    (; mc, me) = params
    return mc .* (grid .^ me)
end

matching_function (generic function with 1 method)

In [29]:
function generate_income_process(params)
    (; n, lb, ub) = params
    Δ = ub / n
    grid = LinRange(lb, ub, n)
    exp_grid = exp.(grid)
    transition_matrix = zeros(n, n)
    for i in 1:n
        if i == 1
            transition_matrix[i, i] = ( 1/2 ) * (1 .+ (grid[i] / (n*Δ)))
            transition_matrix[i, i+1] = ( 1/2 ) * (1 .- (grid[i] / (n*Δ)))
        elseif i == n
            transition_matrix[i, i-1] = ( 1/2 ) * (1 .+ (grid[i] / (n*Δ)))
            transition_matrix[i, i] = ( 1/2 ) * (1 .- (grid[i] / (n*Δ)))
        else
            transition_matrix[i, i-1] = ( 1/2 ) * (1 .+ (grid[i] / (n*Δ)))
            transition_matrix[i, i+1] = ( 1/2 ) * (1 .- (grid[i] / (n*Δ)))
        end
    end
    return exp_grid, transition_matrix
end

income_process, transition_matrix = generate_income_process(params)

([0.36787944117144233, 0.3682475046136629, 0.3686159363034188, 0.3689847366091417, 0.369353905899632, 0.3697234445440589, 0.3700933529119613, 0.37046363137324734, 0.37083428029819565, 0.37120530005745517  …  2.6939270528874992, 2.696622327353013, 2.6993202984410796, 2.7020209688496686, 2.7047243412794524, 2.7074304184338023, 2.7101392030187967, 2.7128506977432196, 2.715564905318567, 2.718281828459045], [0.0 1.0 … 0.0 0.0; 0.0004999999999999449 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0004999999999999449; 0.0 0.0 … 1.0 0.0])

In [49]:
function productivity_grid(params)
    (; z, p_star) = params
    grid = generate_income_process(params)
    p_grid = z .+ (grid[1] .* (p_star - z))
    return p_grid
end

function separation_grid(params)
    (; n, s_star) = params
    grid = generate_income_process(params)
    s_grid = grid[1] .* s_star
    return s_grid
end

function theta_grid(params)
    (; n) = params
    grid = generate_income_process(params)
    θ_grid = LinRange(1,2,n)
    return θ_grid
end
    

theta_grid (generic function with 1 method)

In [57]:
function iterate(grid, params)
    (; β, r, λ, z, p_star, s_star, c, σ, γ, n, lb, ub, mc, me, max_iter) = params
    Δ = ub / n
    exp_grid, transition_matrix = generate_income_process(params)
    prod_grid = productivity_grid(params)
    match_grid = matching_function(grid, params)
    iter = 0
    error = Inf
    new_grid = ones(n)
    while iter < max_iter && error > 1e-6
        iter += 1
        for i in 1:n
            if i == 1
                new_grid[i] = optimize(theta -> - ((r+s_star+λ) / theta) + (((1-β)*(prod_grid[i]-z)) / c ) - β*theta + λ*((transition_matrix[i, i+1] / grid[i+1])), 0, 2).minimizer
            elseif i == n
                new_grid[i] = optimize(theta -> - ((r+s_star+λ) / theta) + (((1-β)*(prod_grid[i]-z)) / c ) - β*theta + λ*((transition_matrix[i, i-1] / grid[i-1])), 0, 2).minimizer
            else
                new_grid[i] = optimize(theta -> - ((r+s_star+λ) / theta) + (((1-β)*(prod_grid[i]-z)) / c ) - β*theta + λ*((transition_matrix[i, i-1] / grid[i-1]) + (transition_matrix[i, i+1] / grid[i+1])), 0, 2).minimizer
            end
        end
        error = norm(new_grid - grid)
        grid = new_grid
        if iter % 1 == 0
            println("Iteration: $iter, Error: $error")
        end
    end
    return grid
end

iterate (generic function with 1 method)

In [58]:
θ_grid = theta_grid(params)
thetas = iterate(θ_grid, params)    

Iteration: 1, Error: 68.3313013925536
Iteration: 2, Error: 0.0


2001-element Vector{Float64}:
 4.2360100022891096e-16
 4.2360100022891096e-16
 4.2360100022891096e-16
 4.2360100022891096e-16
 4.2360100022891096e-16
 4.2360100022891096e-16
 4.2360100022891096e-16
 4.2360100022891096e-16
 4.2360100022891096e-16
 4.2360100022891096e-16
 ⋮
 4.2360100022891096e-16
 4.2360100022891096e-16
 4.2360100022891096e-16
 4.2360100022891096e-16
 4.2360100022891096e-16
 4.2360100022891096e-16
 4.2360100022891096e-16
 4.2360100022891096e-16
 4.2360100022891096e-16